## How is the dispersal rate estimation by BEAST affected by sampling area and dispersal model?

This notebook investigates how the **sampling area** and the **underlying dispersal model** influence the accuracy and variability of dispersal rate estimates inferred by BEAST using continuous phylogeographic diffusion models.

---

### 1. GSpace simulation using various sampling areas

We simulate sequence data under controlled spatial population genetic conditions using GSpace. The key variable in this notebook is the **sampling radius** \( r \), which defines the sampling area.

#### Sampling area:

The sampling area is defined as a square located at the center of the lattice with area:

\[
A = 2r^2
\]

- A larger \( r \) results in a broader spatial coverage.
- A smaller \( r \) means more spatially restricted sampling.

We vary \( r \in \{2, 4, 6, 8\} \), keeping the total number of sampled individuals constant across runs.

#### Parameters (fixed):

- Lattice size: 20 × 20
- Individuals per node: 30
- Sampled individuals: 20 (5 per node)
- Chromosome: 1, 1000 bp
- Mutation model: HKY
- Dispersal model: uniform, maximum distance = 1
- Output: FASTA + coordinates in header

#### Objective:

To assess whether **increasing the sampling area**:

- Increases the precision or bias of **diffusion rate estimates**.
- Improves the correlation between true and estimated movement distances.